In [10]:
# 네이버 데이터랩에서 검색어 추이 분석(검색어 최대 5개 입력받을 수 있음)

import requests
import pandas as pd
import matplotlib.pyplot as plt
from tkinter import Tk, Toplevel, Label, Button, messagebox, ttk, Entry
from tkcalendar import Calendar
import tkinter as tk
from deep_translator import GoogleTranslator
import json
from datetime import datetime, timedelta

# 네이버 API 인증 정보 (하드코딩)
client_id = 'NM0_03gTtgRgamfzdgpM'
client_secret = 'wVJP2iAT52'

if not client_id or not client_secret:
    raise ValueError("NAVER_CLIENT_ID와 NAVER_CLIENT_SECRET이 설정되지 않았습니다.")

class NaverSearchTrendApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Naver Search Trends")
        self.root.geometry("500x200")

        self.keywords = []
        self.start_date = None
        self.end_date = None

        self.setup_ui()

    def setup_ui(self):
        # 검색어 입력 라벨
        keyword_label = Label(self.root, text="검색어 입력 :")
        keyword_label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        # 검색어 입력 필드
        self.keyword_entry = Entry(self.root, width=40)
        self.keyword_entry.grid(row=0, column=1, padx=10, pady=10, columnspan=3)

        # 시작 날짜 선택 버튼
        self.start_date_button = Button(self.root, text="시작 날짜 선택", command=self.get_start_date)
        self.start_date_button.grid(row=1, column=0, padx=10, pady=10)

        # 종료 날짜 선택 버튼
        self.end_date_button = Button(self.root, text="종료 날짜 선택", command=self.get_end_date)
        self.end_date_button.grid(row=1, column=1, padx=10, pady=10)

        # 검색 버튼
        self.search_button = Button(self.root, text="검색 시작", command=self.perform_search)
        self.search_button.grid(row=1, column=2, padx=10, pady=10)

        # 종료 버튼
        self.quit_button = Button(self.root, text="프로그램 종료", command=self.quit_program)
        self.quit_button.grid(row=1, column=3, padx=10, pady=10)

    def get_date(self, is_start=True):
        def on_date_selected():
            selected_date = cal.selection_get().strftime('%Y-%m-%d')
            if is_start:
                self.start_date = selected_date
            else:
                self.end_date = selected_date
            date_window.destroy()

        date_window = Toplevel(self.root)
        date_window.title("날짜 선택")
        date_window.geometry("400x300")

        Label(date_window, text="날짜 선택").pack(pady=5)
        cal = Calendar(date_window, selectmode='day', date_pattern='yyyy-mm-dd')
        cal.pack(pady=10)

        Button(date_window, text="확인", command=on_date_selected).pack(pady=20)

    def get_start_date(self):
        self.get_date(is_start=True)

    def get_end_date(self):
        self.get_date(is_start=False)

    def get_default_dates(self):
        if not self.start_date:
            self.start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')
        if not self.end_date:
            self.end_date = datetime.today().strftime('%Y-%m-%d')

    def perform_search(self):
        # 기본 날짜 설정
        self.get_default_dates()

        keywords_str = self.keyword_entry.get()
        if keywords_str:
            self.keywords = [k.strip() for k in keywords_str.split(",") if k.strip()]
            if len(self.keywords) > 5:
                messagebox.showwarning("입력 오류", "최대 5개의 검색어만 입력할 수 있습니다.")
                self.keywords = self.keywords[:5]
        else:
            messagebox.showwarning("입력 오류", "검색어를 입력하세요.")
            self.keyword_entry.focus()
            return

        try:
            # 번역기 초기화 및 검색어 번역
            translated_keywords = [GoogleTranslator(source='auto', target='en').translate(keyword) for keyword in self.keywords]
        except Exception as e:
            messagebox.showwarning("번역 오류", f"번역 중 오류가 발생했습니다: {e}")
            translated_keywords = self.keywords

        # API 요청 URL 및 헤더 설정
        url = 'https://openapi.naver.com/v1/datalab/search'
        headers = {
            'X-Naver-Client-Id': client_id,
            'X-Naver-Client-Secret': client_secret,
            'Content-Type': 'application/json'
        }

        # 요청 데이터 설정
        keyword_groups = [{"groupName": keyword, "keywords": [keyword]} for keyword in self.keywords]
        data = {
            "startDate": self.start_date,
            "endDate": self.end_date,
            "timeUnit": "month",
            "keywordGroups": keyword_groups
        }

        # API 요청
        response = requests.post(url, headers=headers, json=data)

        # 응답 데이터 확인 및 오류 출력
        if response.status_code == 200:
            result = response.json()
            self.show_results(result, translated_keywords)
        elif response.status_code == 400:
            messagebox.showerror("Error Code: 400 - Bad Request", response.content.decode('utf-8'))
        else:
            messagebox.showerror(f"Error Code: {response.status_code}", response.content.decode('utf-8'))

    def show_results(self, result, translated_keywords):
        # 새로운 윈도우에 그래프 표시
        result_window = Toplevel(self.root)
        result_window.title("검색 결과")
        result_window.geometry("1000x600")

        plt.figure(figsize=(12, 6))

        for keyword_group, translated_keyword in zip(result['results'], translated_keywords):
            df = pd.DataFrame(keyword_group['data'])
            df['period'] = pd.to_datetime(df['period'])
            df.set_index('period', inplace=True)
            plt.plot(df.index, df['ratio'], marker='o', linestyle='-', label=translated_keyword)

        plt.title('Keyword Search Trends')
        plt.xlabel('Date')
        plt.ylabel('Search Ratio')
        plt.legend()
        plt.grid(True)

        # 그래프를 캔버스 위에 표시
        from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
        canvas = FigureCanvasTkAgg(plt.gcf(), master=result_window)
        canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        canvas.draw()

    def quit_program(self):
        self.root.destroy()

if __name__ == "__main__":
    root = Tk()
    app = NaverSearchTrendApp(root)
    root.mainloop()
